# ADL2CSV
+ ADLtagger形式のディレクトリ構造からCSVデータを抜き出し, tag情報を付加したうえでひとつのCSVファイルにまとめて出力する.
+ ラベルデータの変換とCSVの結合を別に行う

### [注意!!] タイムスタンプの桁落ち 必ず3桁にしないとsortした時に変になる.

In [4]:
import os # os.mkdir(path)でdirectoryを作成

import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import datetime as dt

# Main関数: CSVのみを扱う
以下の２つのセルを実行

In [2]:
import os # os.mkdir(path)でdirectoryを作成

import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import datetime as dt


class ADL2Csv_csv_only(object):
    """
    Generate Labeled signal data
    """
    def __init__(self, path_to_csv_root_dir='./data/fromADL/', path_to_output_dir='./data/CSV/', sensor_tag='None', fix_timestamp=False):
        """
        Argument
        --------
        path_to_log_dir: str(path to csv files[ADLtagger format])
        path_to_output_dir: str
        """
        self.path_to_csv_root_dir = path_to_csv_root_dir
        self.path_to_output_dir = path_to_output_dir
        self.sensor_tag = sensor_tag
        self.fix_timestamp = fix_timestamp        
        
        
    """Read Data"""
    def read_data(self):
        print("Step1: Read csv files and label xml data.")
        # CSV files
        df = pd.DataFrame()
        df = self.BFS(df, self.path_to_csv_root_dir)
        #  timestampの桁落ちを修正
        print("[Read Data] fix timestamp = ", self.fix_timestamp)
        if self.fix_timestamp:
            df_tmp = df[["time"]].copy()
            df_tmp["time_1"] = "none"
            df_tmp["time_2"] = "none"
            df_tmp[["time_1","time_2"]] = df_tmp["time"].str.split(".", expand=True)
            df["time"] = df_tmp["time_1"] + "." +df_tmp["time_2"].str.zfill(3)
        self.df = df.sort_values(by=["time"]).reset_index(drop=True)
        print(">> Success: df="+str(self.df.shape)+"\n")
        return self.df
        
    ## CSVファイルを取得 ##
    def BFS(self, df, current_path):
        # 現在のディレクトリにあるディレクトリを取得
        for x in os.listdir(current_path):
            print(">>",current_path+x)
            if os.path.isdir(current_path + x):
                # ディレクトリが存在する場合 ==> より深く潜る
                df = self.BFS(df, current_path + x + '/')
            elif os.path.isfile(current_path + x):
                # ファイル場合 ==> CSVか判定 ==> YES: dfに追加
                root, ext = os.path.splitext(x)
                if ext == ".csv":
                    df = pd.concat([df, pd.read_csv(current_path+x)])
        return df
    
    
    """CSV書き出し"""
    def to_csv(self):
        print("Step3: Write CSV files.")
        # ディレクトリを作成
        df, output_path = self.df, self.path_to_output_dir
        if not os.path.isdir(output_path):
            os.mkdir(output_path)
        df.to_csv(output_path+self.sensor_tag+"_labeled.csv")
        print(">> Success: Write csv.[", output_path+self.sensor_tag+"_labeled.csv", "]")
    
# converter = ADL2Csv(path_to_xml='./data/2017_09_21/03_fromADL/data.xml',
#                     path_to_csv_root_dir='./data/2017_09_21/03_fromADL/data/acc2/',
#                     path_to_output_dir='./data/2017_09_21/04_Labeled/',
#                    sensor_tag="acc")
# df, df_label = converter.read_data()
# df = converter.add_label()
# converter.to_csv()
# display(df.head(10))
#display(df_label.head(10))

In [6]:

root_dir = "/root/upconversion/data/2018_01_16/08_higashide"


converter = ADL2Csv_csv_only(
                    path_to_csv_root_dir=root_dir+"/data/acc2/",
                    path_to_output_dir='./data/2018_01_16/',
                   sensor_tag="acc",
                   fix_timestamp=False)
df = converter.read_data()
converter.to_csv()
display(df.head(10))

Step1: Read csv files and label xml data.
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_160700_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_160100_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_160900_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_161000_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_160300_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_160000_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_155500_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_155900_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_160200_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/da

,time,x,y,z
0,20180116_15:55:11.987,445,-1690,9944
1,20180116_15:55:11.988,484,-1704,9964
2,20180116_15:55:11.989,484,-1694,9966
3,20180116_15:55:11.990,472,-1673,9949
4,20180116_15:55:11.991,457,-1658,9969
5,20180116_15:55:11.992,455,-1629,9978
6,20180116_15:55:11.993,482,-1619,9961
7,20180116_15:55:11.994,470,-1660,9944
8,20180116_15:55:11.995,482,-1648,9964
9,20180116_15:55:11.996,523,-1609,10000


In [9]:
""" 2018.01.16
"""
import pandas as pd

sensor = "acc"
name_list = [
    "01_kawabe", "02_higashinaka", "03_yasuda", "04_kamiya",
    "05_ishiyama", "06_yoshimura", "07_aiko", "08_higashide",
]


for _name in name_list[:]:
    print(">> ", _name)
    root_dir = "/root/upconversion/data/2018_01_16/{}".format(_name)
    #df_raw = pd.read_csv(root_dir+"/data/Labeled/{}_labeled.csv".format(sensor), index_col=0)
    
    converter = ADL2Csv_csv_only(
        path_to_csv_root_dir=root_dir+"/data/acc2/",
        path_to_output_dir=root_dir+"/data/Labeled/",
        sensor_tag=sensor,
        fix_timestamp=False)
    df = converter.read_data()
    converter.to_csv()
    display(df.head(10))

>>  01_kawabe
Step1: Read csv files and label xml data.
>> /root/upconversion/data/2018_01_16/01_kawabe/data/acc2/acc2_R
>> /root/upconversion/data/2018_01_16/01_kawabe/data/acc2/acc2_R/20180116_131100_acc2.csv
>> /root/upconversion/data/2018_01_16/01_kawabe/data/acc2/acc2_R/20180116_130900_acc2.csv
>> /root/upconversion/data/2018_01_16/01_kawabe/data/acc2/acc2_R/20180116_131400_acc2.csv
>> /root/upconversion/data/2018_01_16/01_kawabe/data/acc2/acc2_R/20180116_130100_acc2.csv
>> /root/upconversion/data/2018_01_16/01_kawabe/data/acc2/acc2_R/20180116_132400_acc2.csv
>> /root/upconversion/data/2018_01_16/01_kawabe/data/acc2/acc2_R/20180116_130500_acc2.csv
>> /root/upconversion/data/2018_01_16/01_kawabe/data/acc2/acc2_R/20180116_131700_acc2.csv
>> /root/upconversion/data/2018_01_16/01_kawabe/data/acc2/acc2_R/20180116_130000_acc2.csv
>> /root/upconversion/data/2018_01_16/01_kawabe/data/acc2/acc2_R/20180116_132200_acc2.csv
>> /root/upconversion/data/2018_01_16/01_kawabe/data/acc2/acc2_R/2018

,time,x,y,z
0,20180116_12:59:16.157,2357,-1072,9981
1,20180116_12:59:16.158,2347,-1065,9971
2,20180116_12:59:16.159,2381,-1087,9969
3,20180116_12:59:16.160,2428,-1091,10000
4,20180116_12:59:16.161,2440,-1050,10018
5,20180116_12:59:16.162,2440,-1045,9993
6,20180116_12:59:16.163,2447,-1035,9981
7,20180116_12:59:16.164,2479,-1011,9935
8,20180116_12:59:16.165,2513,-989,9893
9,20180116_12:59:16.166,2537,-994,9942


>>  02_higashinaka
Step1: Read csv files and label xml data.
>> /root/upconversion/data/2018_01_16/02_higashinaka/data/acc2/acc2_R
>> /root/upconversion/data/2018_01_16/02_higashinaka/data/acc2/acc2_R/20180116_132400_acc2.csv
>> /root/upconversion/data/2018_01_16/02_higashinaka/data/acc2/acc2_R/20180116_133100_acc2.csv
>> /root/upconversion/data/2018_01_16/02_higashinaka/data/acc2/acc2_R/20180116_133500_acc2.csv
>> /root/upconversion/data/2018_01_16/02_higashinaka/data/acc2/acc2_R/20180116_132200_acc2.csv
>> /root/upconversion/data/2018_01_16/02_higashinaka/data/acc2/acc2_R/20180116_133400_acc2.csv
>> /root/upconversion/data/2018_01_16/02_higashinaka/data/acc2/acc2_R/20180116_133000_acc2.csv
>> /root/upconversion/data/2018_01_16/02_higashinaka/data/acc2/acc2_R/20180116_133200_acc2.csv
>> /root/upconversion/data/2018_01_16/02_higashinaka/data/acc2/acc2_R/20180116_132700_acc2.csv
>> /root/upconversion/data/2018_01_16/02_higashinaka/data/acc2/acc2_R/20180116_132600_acc2.csv
>> /root/upcon

,time,x,y,z
0,20180116_13:20:49.887,-93,907,10174
1,20180116_13:20:49.888,-120,909,10176
2,20180116_13:20:49.889,-147,929,10174
3,20180116_13:20:49.890,-142,939,10203
4,20180116_13:20:49.891,-152,917,10205
5,20180116_13:20:49.892,-220,902,10164
6,20180116_13:20:49.893,-269,905,10176
7,20180116_13:20:49.894,-264,907,10208
8,20180116_13:20:49.895,-242,944,10198
9,20180116_13:20:49.896,-230,934,10196


>>  03_yasuda
Step1: Read csv files and label xml data.
>> /root/upconversion/data/2018_01_16/03_yasuda/data/acc2/acc2_R
>> /root/upconversion/data/2018_01_16/03_yasuda/data/acc2/acc2_R/20180116_115100_acc2.csv
>> /root/upconversion/data/2018_01_16/03_yasuda/data/acc2/acc2_R/20180116_120500_acc2.csv
>> /root/upconversion/data/2018_01_16/03_yasuda/data/acc2/acc2_R/20180116_115600_acc2.csv
>> /root/upconversion/data/2018_01_16/03_yasuda/data/acc2/acc2_R/20180116_120200_acc2.csv
>> /root/upconversion/data/2018_01_16/03_yasuda/data/acc2/acc2_R/20180116_115800_acc2.csv
>> /root/upconversion/data/2018_01_16/03_yasuda/data/acc2/acc2_R/20180116_121000_acc2.csv
>> /root/upconversion/data/2018_01_16/03_yasuda/data/acc2/acc2_R/20180116_120400_acc2.csv
>> /root/upconversion/data/2018_01_16/03_yasuda/data/acc2/acc2_R/20180116_120100_acc2.csv
>> /root/upconversion/data/2018_01_16/03_yasuda/data/acc2/acc2_R/20180116_115400_acc2.csv
>> /root/upconversion/data/2018_01_16/03_yasuda/data/acc2/acc2_R/2018

...

,time,x,y,z
0,20180116_14:30:30.577,-220,-237,9981
1,20180116_14:30:30.578,-218,-266,9959
2,20180116_14:30:30.579,-208,-257,9932
3,20180116_14:30:30.580,-225,-249,9956
4,20180116_14:30:30.581,-218,-269,9959
5,20180116_14:30:30.582,-220,-252,9920
6,20180116_14:30:30.583,-198,-252,9939
7,20180116_14:30:30.584,-186,-264,9961
8,20180116_14:30:30.585,-223,-259,9947
9,20180116_14:30:30.586,-255,-220,9964


>>  07_aiko
Step1: Read csv files and label xml data.
>> /root/upconversion/data/2018_01_16/07_aiko/data/acc2/acc2_R
>> /root/upconversion/data/2018_01_16/07_aiko/data/acc2/acc2_R/20180116_170500_acc2.csv
>> /root/upconversion/data/2018_01_16/07_aiko/data/acc2/acc2_R/20180116_165100_acc2.csv
>> /root/upconversion/data/2018_01_16/07_aiko/data/acc2/acc2_R/20180116_170900_acc2.csv
>> /root/upconversion/data/2018_01_16/07_aiko/data/acc2/acc2_R/20180116_165600_acc2.csv
>> /root/upconversion/data/2018_01_16/07_aiko/data/acc2/acc2_R/20180116_171000_acc2.csv
>> /root/upconversion/data/2018_01_16/07_aiko/data/acc2/acc2_R/20180116_165800_acc2.csv
>> /root/upconversion/data/2018_01_16/07_aiko/data/acc2/acc2_R/20180116_170400_acc2.csv
>> /root/upconversion/data/2018_01_16/07_aiko/data/acc2/acc2_R/20180116_170100_acc2.csv
>> /root/upconversion/data/2018_01_16/07_aiko/data/acc2/acc2_R/20180116_171100_acc2.csv
>> /root/upconversion/data/2018_01_16/07_aiko/data/acc2/acc2_R/20180116_165500_acc2.csv
>> 

,time,x,y,z
0,20180116_16:50:05.147,-838,-506,9930
1,20180116_16:50:05.148,-828,-525,9898
2,20180116_16:50:05.149,-816,-493,9937
3,20180116_16:50:05.150,-843,-488,9937
4,20180116_16:50:05.151,-882,-510,9888
5,20180116_16:50:05.152,-897,-479,9917
6,20180116_16:50:05.153,-894,-462,9922
7,20180116_16:50:05.154,-872,-474,9922
8,20180116_16:50:05.155,-870,-466,9961
9,20180116_16:50:05.156,-884,-486,9971


>>  08_higashide
Step1: Read csv files and label xml data.
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_160700_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_160100_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_160900_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_161000_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_160300_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_160000_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_155500_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_155900_acc2.csv
>> /root/upconversion/data/2018_01_16/08_higashide/data/acc2/acc2_R/20180116_160200_acc2.csv
>> /root/upconversion/data/2018_01_1

,time,x,y,z
0,20180116_15:55:11.987,445,-1690,9944
1,20180116_15:55:11.988,484,-1704,9964
2,20180116_15:55:11.989,484,-1694,9966
3,20180116_15:55:11.990,472,-1673,9949
4,20180116_15:55:11.991,457,-1658,9969
5,20180116_15:55:11.992,455,-1629,9978
6,20180116_15:55:11.993,482,-1619,9961
7,20180116_15:55:11.994,470,-1660,9944
8,20180116_15:55:11.995,482,-1648,9964
9,20180116_15:55:11.996,523,-1609,10000


In [5]:
""" 2018.01.18
"""

converter = ADL2Csv_csv_only(
                    path_to_csv_root_dir="./data/ADL/acc2/",
                    path_to_output_dir='./data/CSV/',
                   sensor_tag="acc",
                   fix_timestamp=False)
df = converter.read_data()
converter.to_csv()
display(df.head(10))

Step1: Read csv files and label xml data.
>> ./data/ADL/acc2/acc2_R
>> ./data/ADL/acc2/acc2_R/20170921_174400_acc2.csv
>> ./data/ADL/acc2/acc2_R/20170921_174500_acc2.csv
>> ./data/ADL/acc2/acc2_R/20170921_174600_acc2.csv
>> ./data/ADL/acc2/acc2_R/20170921_174700_acc2.csv
[Read Data] fix timestamp =  False
>> Success: df=(180760, 4)

Step3: Write CSV files.
>> Success: Write csv.[ ./data/CSV/acc_labeled.csv ]


,time,x,y,z
0,20170921_17:44:12.483,1497,-77,10306
1,20170921_17:44:12.484,1514,-79,10291
2,20170921_17:44:12.485,1527,-74,10284
3,20170921_17:44:12.486,1517,-40,10259
4,20170921_17:44:12.487,1568,-23,10235
5,20170921_17:44:12.488,1597,-26,10276
6,20170921_17:44:12.489,1583,-38,10296
7,20170921_17:44:12.490,1580,-35,10237
8,20170921_17:44:12.491,1602,-21,10196
9,20170921_17:44:12.492,1612,9,10196


In [3]:
""" 2018.02.01
"""
import pandas as pd

sensor = "acc"
name_list = [
    #"30_hada", "31_teramae","32_torisuke", 
    "33_matsukawa",
    #"34_sato","35_hamase","36_yamasaki",
    "37_koguchi","38_onuma","39_kashiyama","40_yamaguchi","41_watase","42_oga","43_shigeyoshi","44_hukuda","45_maekawa",
]


for _name in name_list[:]:
    print(">> ", _name)
    root_dir = "/root/upconversion/data/2018_01_16/{}".format(_name)
    #df_raw = pd.read_csv(root_dir+"/data/Labeled/{}_labeled.csv".format(sensor), index_col=0)
    
    converter = ADL2Csv_csv_only(
        path_to_csv_root_dir=root_dir+"/data/acc2/",
        path_to_output_dir=root_dir+"/data/Labeled/",
        sensor_tag=sensor,
        fix_timestamp=False)
    df = converter.read_data()
    converter.to_csv()
    display(df.head(10))

>>  33_matsukawa
Step1: Read csv files and label xml data.
>> /root/upconversion/data/2018_01_16/33_matsukawa/data/acc2/acc2_R
>> /root/upconversion/data/2018_01_16/33_matsukawa/data/acc2/acc2_R/20180131_214800_acc2.csv
>> /root/upconversion/data/2018_01_16/33_matsukawa/data/acc2/acc2_R/20180131_214000_acc2.csv
>> /root/upconversion/data/2018_01_16/33_matsukawa/data/acc2/acc2_R/20180131_214700_acc2.csv
>> /root/upconversion/data/2018_01_16/33_matsukawa/data/acc2/acc2_R/20180131_215000_acc2.csv
>> /root/upconversion/data/2018_01_16/33_matsukawa/data/acc2/acc2_R/20180131_214100_acc2.csv
>> /root/upconversion/data/2018_01_16/33_matsukawa/data/acc2/acc2_R/20180131_214400_acc2.csv
>> /root/upconversion/data/2018_01_16/33_matsukawa/data/acc2/acc2_R/20180131_214500_acc2.csv
>> /root/upconversion/data/2018_01_16/33_matsukawa/data/acc2/acc2_R/20180131_214200_acc2.csv
>> /root/upconversion/data/2018_01_16/33_matsukawa/data/acc2/acc2_R/20180131_214900_acc2.csv
>> /root/upconversion/data/2018_01_1

,time,x,y,z
0,20180131_21:40:24.491,548,1574,10279
1,20180131_21:40:24.492,557,1603,10205
2,20180131_21:40:24.493,589,1566,10157
3,20180131_21:40:24.494,621,1544,10169
4,20180131_21:40:24.495,628,1500,10203
5,20180131_21:40:24.496,606,1488,10196
6,20180131_21:40:24.497,609,1513,10203
7,20180131_21:40:24.498,631,1456,10164
8,20180131_21:40:24.499,636,1398,10093
9,20180131_21:40:24.500,616,1408,10091


>>  37_koguchi
Step1: Read csv files and label xml data.
>> /root/upconversion/data/2018_01_16/37_koguchi/data/acc2/acc2_R
>> /root/upconversion/data/2018_01_16/37_koguchi/data/acc2/acc2_R/20180201_162800_acc2.csv
>> /root/upconversion/data/2018_01_16/37_koguchi/data/acc2/acc2_R/20180201_162300_acc2.csv
>> /root/upconversion/data/2018_01_16/37_koguchi/data/acc2/acc2_R/20180201_163400_acc2.csv
>> /root/upconversion/data/2018_01_16/37_koguchi/data/acc2/acc2_R/20180201_162600_acc2.csv
>> /root/upconversion/data/2018_01_16/37_koguchi/data/acc2/acc2_R/20180201_162400_acc2.csv
>> /root/upconversion/data/2018_01_16/37_koguchi/data/acc2/acc2_R/20180201_162900_acc2.csv
>> /root/upconversion/data/2018_01_16/37_koguchi/data/acc2/acc2_R/20180201_163300_acc2.csv
>> /root/upconversion/data/2018_01_16/37_koguchi/data/acc2/acc2_R/20180201_163100_acc2.csv
>> /root/upconversion/data/2018_01_16/37_koguchi/data/acc2/acc2_R/20180201_163200_acc2.csv
>> /root/upconversion/data/2018_01_16/37_koguchi/data/acc2

,time,x,y,z
0,20180201_16:23:46.821,174,988,10249
1,20180201_16:23:46.822,179,992,10254
2,20180201_16:23:46.823,116,1000,10203
3,20180201_16:23:46.824,91,1012,10132
4,20180201_16:23:46.825,79,1010,10105
5,20180201_16:23:46.826,58,973,10132
6,20180201_16:23:46.827,75,978,10132
7,20180201_16:23:46.828,86,992,10132
8,20180201_16:23:46.829,108,1000,10149
9,20180201_16:23:46.830,125,988,10140


>>  38_onuma
Step1: Read csv files and label xml data.
>> /root/upconversion/data/2018_01_16/38_onuma/data/acc2/acc2_R
>> /root/upconversion/data/2018_01_16/38_onuma/data/acc2/acc2_R/20180203_135300_acc2.csv
>> /root/upconversion/data/2018_01_16/38_onuma/data/acc2/acc2_R/20180203_135100_acc2.csv
>> /root/upconversion/data/2018_01_16/38_onuma/data/acc2/acc2_R/20180203_134400_acc2.csv
>> /root/upconversion/data/2018_01_16/38_onuma/data/acc2/acc2_R/20180203_134900_acc2.csv
>> /root/upconversion/data/2018_01_16/38_onuma/data/acc2/acc2_R/20180203_134500_acc2.csv
>> /root/upconversion/data/2018_01_16/38_onuma/data/acc2/acc2_R/20180203_135200_acc2.csv
>> /root/upconversion/data/2018_01_16/38_onuma/data/acc2/acc2_R/20180203_134600_acc2.csv
>> /root/upconversion/data/2018_01_16/38_onuma/data/acc2/acc2_R/20180203_134700_acc2.csv
>> /root/upconversion/data/2018_01_16/38_onuma/data/acc2/acc2_R/20180203_135000_acc2.csv
>> /root/upconversion/data/2018_01_16/38_onuma/data/acc2/acc2_R/20180203_134800_

...

,time,x,y,z
0,20180203_15:45:23.756,-1114,-4446,8953
1,20180203_15:45:23.757,-1129,-4470,8965
2,20180203_15:45:23.758,-1129,-4492,8934
3,20180203_15:45:23.759,-1153,-4483,8970
4,20180203_15:45:23.760,-1141,-4475,9024
5,20180203_15:45:23.761,-1163,-4426,8992
6,20180203_15:45:23.762,-1226,-4390,8973
7,20180203_15:45:23.763,-1253,-4392,8970
8,20180203_15:45:23.764,-1265,-4329,9019
9,20180203_15:45:23.765,-1302,-4238,9085


>>  44_hukuda
Step1: Read csv files and label xml data.
>> /root/upconversion/data/2018_01_16/44_hukuda/data/acc2/acc2_R
>> /root/upconversion/data/2018_01_16/44_hukuda/data/acc2/acc2_R/20180203_162900_acc2.csv
>> /root/upconversion/data/2018_01_16/44_hukuda/data/acc2/acc2_R/20180203_163000_acc2.csv
>> /root/upconversion/data/2018_01_16/44_hukuda/data/acc2/acc2_R/20180203_162200_acc2.csv
>> /root/upconversion/data/2018_01_16/44_hukuda/data/acc2/acc2_R/20180203_162300_acc2.csv
>> /root/upconversion/data/2018_01_16/44_hukuda/data/acc2/acc2_R/20180203_162100_acc2.csv
>> /root/upconversion/data/2018_01_16/44_hukuda/data/acc2/acc2_R/20180203_162500_acc2.csv
>> /root/upconversion/data/2018_01_16/44_hukuda/data/acc2/acc2_R/20180203_162000_acc2.csv
>> /root/upconversion/data/2018_01_16/44_hukuda/data/acc2/acc2_R/20180203_162800_acc2.csv
>> /root/upconversion/data/2018_01_16/44_hukuda/data/acc2/acc2_R/20180203_161800_acc2.csv
>> /root/upconversion/data/2018_01_16/44_hukuda/data/acc2/acc2_R/2018

,time,x,y,z
0,20180203_16:17:12.416,-130,-813,9759
1,20180203_16:17:12.417,-132,-828,9788
2,20180203_16:17:12.418,-159,-803,9795
3,20180203_16:17:12.419,-179,-777,9825
4,20180203_16:17:12.420,-240,-764,9861
5,20180203_16:17:12.421,-294,-713,9864
6,20180203_16:17:12.422,-316,-684,9871
7,20180203_16:17:12.423,-357,-698,9922
8,20180203_16:17:12.424,-386,-686,9959
9,20180203_16:17:12.425,-386,-681,9939


>>  45_maekawa
Step1: Read csv files and label xml data.
>> /root/upconversion/data/2018_01_16/45_maekawa/data/acc2/acc2_R
>> /root/upconversion/data/2018_01_16/45_maekawa/data/acc2/acc2_R/20180204_122200_acc2.csv
>> /root/upconversion/data/2018_01_16/45_maekawa/data/acc2/acc2_R/20180204_122300_acc2.csv
>> /root/upconversion/data/2018_01_16/45_maekawa/data/acc2/acc2_R/20180204_121300_acc2.csv
>> /root/upconversion/data/2018_01_16/45_maekawa/data/acc2/acc2_R/20180204_121700_acc2.csv
>> /root/upconversion/data/2018_01_16/45_maekawa/data/acc2/acc2_R/20180204_122500_acc2.csv
>> /root/upconversion/data/2018_01_16/45_maekawa/data/acc2/acc2_R/20180204_121500_acc2.csv
>> /root/upconversion/data/2018_01_16/45_maekawa/data/acc2/acc2_R/20180204_121900_acc2.csv
>> /root/upconversion/data/2018_01_16/45_maekawa/data/acc2/acc2_R/20180204_122400_acc2.csv
>> /root/upconversion/data/2018_01_16/45_maekawa/data/acc2/acc2_R/20180204_122000_acc2.csv
>> /root/upconversion/data/2018_01_16/45_maekawa/data/acc2

,time,x,y,z
0,20180204_12:13:12.341,41,121,9971
1,20180204_12:13:12.342,70,128,9944
2,20180204_12:13:12.343,75,121,9986
3,20180204_12:13:12.344,70,123,9983
4,20180204_12:13:12.345,58,153,9915
5,20180204_12:13:12.346,94,170,9913
6,20180204_12:13:12.347,125,189,9920
7,20180204_12:13:12.348,120,197,9898
8,20180204_12:13:12.349,138,197,9854
9,20180204_12:13:12.350,133,194,9854


# Main関数: ラベルデータdata.xml用

In [5]:
import os # os.mkdir(path)でdirectoryを作成

import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import datetime as dt


class ADL2Csv_label_only(object):
    """
    Generate Labeled signal data
    """
    def __init__(self, path_to_xml, path_to_output_dir='./data/LabeledCSV/', sensor_tag='None'):
        """
        Argument
        --------
        path_to_xml: str(Labele data file generated by ADLtagger)
        path_to_log_dir: str(path to csv files[ADLtagger format])
        path_to_output_dir: str
        """
        self.path_to_xml = path_to_xml
        self.path_to_output_dir = path_to_output_dir
        self.sensor_tag = sensor_tag
        
        
    """Read Data"""
    def read_data(self):
        print("Step1: Read csv files and label xml data.")
        # Label
        self.df_label = self.get_labels()
        print(">> Success: df_label="+str(self.df_label.shape))
        return self.df_label
        
    ## XMLの解析 ##
    def get_labels(self):
        def parse_time_str(s):
            try:
                ymd, HMS, ms = s[0:10].replace('-', ''), s[11:19], s[20:]
            except IndexError as e:
                ymd, HMS, ms = s[0:10].replace('-', ''), s[11:19], '000'
            return str(ymd)+'_'+str(HMS)+'.'+str(ms).ljust(3, '0')
        # XML
        tree = ET.parse(self.path_to_xml)
        elem = tree.getroot()
        df_label = []
        for e in elem.findall("labellist"):
            label = e.find("eventtype").text
            start = parse_time_str(e.find("start").text)
            end = parse_time_str(e.find("end").text)
            df_label.append([label, start, end])
        return pd.DataFrame(df_label, columns=["label", "start", "end"])

    """CSV書き出し"""
    def to_csv(self):
        print("Step3: Write CSV files.")
        # ディレクトリを作成
        df_label, output_path = self.df_label, self.path_to_output_dir
        if not os.path.isdir(output_path):
            os.mkdir(output_path)
        df_label.to_csv(output_path+"label.csv")
        print(">> Success: Write csv.[", output_path+"labele.csv", "]")
        df_label[["label", "start"]].groupby(by=["label"]).count().sort_values(["start"], ascending=False).to_csv(output_path+"label_counts.csv")
    
    
# converter = ADL2Csv(path_to_xml='./data/2017_09_21/03_fromADL/data.xml',
#                     path_to_csv_root_dir='./data/2017_09_21/03_fromADL/data/acc2/',
#                     path_to_output_dir='./data/2017_09_21/04_Labeled/',
#                    sensor_tag="acc")
# df, df_label = converter.read_data()
# df = converter.add_label()
# converter.to_csv()
# display(df.head(10))
#display(df_label.head(10))

In [6]:
""" 2017.12.15
"""
import pandas as pd


root_dir = "/root/upconversion/data/2018_01_16/"
sensor = "acc"
# name_list = [
#     "01_kawabe", "02_higashinaka", "03_yasuda", "04_kamiya",
#     "05_ishiyama", "06_yoshimura", "07_aiko", "08_higashide",
# ]
name_list = [
    "30_hada", "32_torisuke", "33_matsukawa", "35_hamase",
    "38_onuma", "39_kashiyama", "44_hukuda",
]


for _name in name_list[:]:
    print(_name)
    root_dir = "/root/upconversion/data/2018_01_16/{}".format(_name)
    #df_raw = pd.read_csv(root_dir+"/data/Labeled/{}_labeled.csv".format(sensor), index_col=0)
    
    converter = ADL2Csv_label_only(path_to_xml=root_dir+"/data.xml",
                        path_to_output_dir=root_dir+"/data/Labeled/",
                        sensor_tag=sensor,
                       )
    df_label = converter.read_data()
    #df = converter.add_label()
    converter.to_csv()
    #display(df.head(10))
    print("\n")

30_hada
Step1: Read csv files and label xml data.
>> Success: df_label=(204, 3)
Step3: Write CSV files.
>> Success: Write csv.[ /root/upconversion/data/2018_01_16/30_hada/data/Labeled/labele.csv ]


32_torisuke
Step1: Read csv files and label xml data.
>> Success: df_label=(201, 3)
Step3: Write CSV files.
>> Success: Write csv.[ /root/upconversion/data/2018_01_16/32_torisuke/data/Labeled/labele.csv ]


33_matsukawa
Step1: Read csv files and label xml data.
>> Success: df_label=(205, 3)
Step3: Write CSV files.
>> Success: Write csv.[ /root/upconversion/data/2018_01_16/33_matsukawa/data/Labeled/labele.csv ]


35_hamase
Step1: Read csv files and label xml data.
>> Success: df_label=(202, 3)
Step3: Write CSV files.
>> Success: Write csv.[ /root/upconversion/data/2018_01_16/35_hamase/data/Labeled/labele.csv ]


38_onuma
Step1: Read csv files and label xml data.
>> Success: df_label=(202, 3)
Step3: Write CSV files.
>> Success: Write csv.[ /root/upconversion/data/2018_01_16/38_onuma/data/Labe